In [28]:
import pandas as pd
import os

df = pd.read_json('../Data/Serie_A_2014_squads.json')

df = df.T
df.head(3)

,0,1,2,3,4,5,6,7,8
1,"{'Home': {'Squad': ['141698', '137644', '14474...","{'Home': {'Squad': ['141698', '137644', '14474...","{'Home': {'Squad': ['141698', '137644', '14474...","{'Home': {'Squad': ['141698', '137644', '14474...","{'Home': {'Squad': ['141698', '137644', '14474...","{'Home': {'Squad': ['141698', '137644', '14474...",NaN,NaN,NaN
2,"{'Home': {'Squad': ['137990', '172996', '16795...","{'Home': {'Squad': ['137990', '172996', '16795...","{'Home': {'Squad': ['137990', '172996', '16795...","{'Home': {'Squad': ['137990', '172996', '16795...","{'Home': {'Squad': ['137990', '172996', '16795...","{'Home': {'Squad': ['137990', '172996', '16795...",NaN,NaN,NaN
3,"{'Home': {'Squad': ['107639', '161940', '45966...","{'Home': {'Squad': ['107639', '161940', '45966...","{'Home': {'Squad': ['107639', '161940', '45966...","{'Home': {'Squad': ['107639', '161940', '31664...",NaN,NaN,NaN,NaN,NaN


Vamos pegar as informações dos ID's dos jogadores com as informações dos Squads. Começaremos pelo campeonato da Série A de 2013 até 2024, e nossa intenção é definir um rating inicial de 1500 para cada jogador.

Com isso, conseguiremos calcular um rating de uma partida e atualizar o rating dos jogadores após ela nesse dataframe com os ID's.

In [5]:

def collect_unique_ids(data_folder, file_names, years):
    """
    Função para coletar IDs únicos de jogadores em arquivos JSON.

    :param data_folder: Caminho para o diretório onde os arquivos JSON estão localizados.
    :param file_names: Lista de nomes de arquivos base (ex: ['Serie_A', 'Serie_B', 'Copa_do_Brasil']).
    :param years: Lista ou range de anos dos arquivos a serem processados.
    :return: Lista de IDs únicos coletados de todos os arquivos.
    """

    unique_ids = set()

    def process_dataframe(df):

        for column in df.columns:
            for cell in df[column]:
                if isinstance(cell, dict):  
                    for key in ['Home', 'Away']:
                        team_info = cell.get(key, {})
                        # Adicionar os IDs do 'Squad'
                        squad_ids = team_info.get('Squad', [])
                        unique_ids.update(squad_ids)


    for file_name in file_names:
        for year in years:
            file_path = os.path.join(data_folder, f'{file_name}_{year}_squads.json')
            if os.path.exists(file_path):
                df = pd.read_json(file_path)
                process_dataframe(df)
            else:
                print(f'Arquivo não encontrado: {file_path}')


    return list(unique_ids)

data_folder = '../Data/'
file_names = ['Serie_A', 'Serie_B', 'Serie_C', 'Serie_D', 'CdB']
years = range(2013, 2025)

unique_player_ids = collect_unique_ids(data_folder, file_names, years)

print(f'Total de IDs únicos coletados: {len(unique_player_ids)}')

Total de IDs únicos coletados: 15448


Total de 15448 diferentes jogadores registrados!

In [33]:
df_players = pd.DataFrame(unique_player_ids, columns=['player'])

# Adicionar a coluna 'rating' com valor inicial de 1500 para cada jogador
df_players['rating'] = 1500

# Mostrar o DataFrame criado
df_players.head()

,player,rating
0,450033,1500
1,188241,1500
2,319850,1500
3,399284,1500
4,566895,1500


In [15]:

df_ = pd.read_json('../Data/Serie_A_2014_games.json')
df_ = df_.T
df_.iloc[1,3]

[['12Diego Diego Cavalieri T(g)P137990', 'Fluminense / RJ'],
 ['2Bruno Bruno Vieira do Nasc ... TP172996', 'Fluminense / RJ'],
 ['3Welington  ... Welington Pereira Ro ... TP167958', 'Fluminense / RJ'],
 ['6Carlinhos Carlos Andrade Souza TP173760', 'Fluminense / RJ'],
 ['7Jean Jean Raphael Vanderl ... TP173180', 'Fluminense / RJ'],
 ['8Rodrigo Ol ... Rodrigo Oliveira de  ... TP147224', 'Fluminense / RJ'],
 ['9Fred Frederico Chaves Guedes TP148994', 'Fluminense / RJ'],
 ['10Wagner Wagner Ferreira dos  ... TP154127', 'Fluminense / RJ'],
 ['11Conca Dario Leonardo Conca TP184395', 'Fluminense / RJ'],
 ['14Elivelton Elivelton Viana dos  ... TP317763', 'Fluminense / RJ'],
 ['23Rafael Sobis Rafael Augusto Sobis TP150549', 'Fluminense / RJ'],
 ['13Chiquinho Francisco Sousa dos  ... RP175503', 'Fluminense / RJ'],
 ['15Rafael Rafael Gimenes da Silva RP307753', 'Fluminense / RJ'],
 ['17Valencia Edwin Armando Valenc ... RP185839', 'Fluminense / RJ'],
 ['18Walter Walter Henrique da Silva RP179832', 

In [37]:
# Função para extrair ID do jogador
def extract_id(player_string):
    match = re.search(r'T\(g\)?P(\d+)|RP(\d+)|TP(\d+)', player_string, re.IGNORECASE)
    if match:
        return next((m for m in match.groups() if m), None)
    return None

# Lista para armazenar os IDs dos jogadores
player_ids = []

# Acessar a linha e coluna específica contendo os dados dos jogadores
player_data = df_.iloc[1, 3]  # Ajuste os índices conforme necessário

# Iterar sobre os dados dos jogadores para extrair os IDs
for player_info in player_data:
    player_string = player_info[0]
    player_id = extract_id(player_string)
    if player_id:
        player_ids.append(player_id)

len(player_ids)

40